In [1]:
import pandas as pd

df = pd.read_csv("../data/performance_data.csv", parse_dates=["date"])
df.head()


,date,service_line,site,incidents,response_time_min,resolution_time_min,sla_target_response_min,sla_target_resolution_min,customer_satisfaction
0,2025-11-01,Health,Adelaide CBD,12,18,240,30,360,4.2
1,2025-11-01,Education,North Adelaide,9,22,310,30,360,4.0
2,2025-11-02,Health,Adelaide CBD,15,28,420,30,360,3.8
3,2025-11-02,Education,North Adelaide,10,19,260,30,360,4.1
4,2025-11-03,Health,Adelaide CBD,11,16,210,30,360,4.3


In [5]:
# Cell 2 — KPI Summary
summary = (
    df.groupby("service_line")
      .agg(
          days_count=("date", "count"),
          total_incidents=("incidents", "sum"),
          avg_response_min=("response_time_min", "mean"),
          avg_resolution_min=("resolution_time_min", "mean"),
          avg_cs=("customer_satisfaction", "mean")
      )
      .round(2)
      .reset_index()
)

summary


,service_line,days_count,total_incidents,avg_response_min,avg_resolution_min,avg_cs
0,Education,5,58,24.4,320.0,3.94
1,Health,5,63,21.4,302.0,4.06


In [6]:
# Cell 3 — SLA Compliance

df["response_sla_met"] = df["response_time_min"] <= df["sla_target_response_min"]
df["resolution_sla_met"] = df["resolution_time_min"] <= df["sla_target_resolution_min"]

sla = (
    df.groupby("service_line")
      .agg(
          response_sla_compliance_pct=("response_sla_met", lambda s: round(100 * s.mean(), 2)),
          resolution_sla_compliance_pct=("resolution_sla_met", lambda s: round(100 * s.mean(), 2)),
      )
      .reset_index()
)

sla


,service_line,response_sla_compliance_pct,resolution_sla_compliance_pct
0,Education,80.0,80.0
1,Health,80.0,60.0


In [7]:
# Cell 4 — Outlier Detection
df["response_vs_sla_pct"] = (100 * df["response_time_min"] / df["sla_target_response_min"]).round(2)

outliers = df[df["response_vs_sla_pct"] > 120].sort_values(
    "response_vs_sla_pct", ascending=False
)

outliers[["date","service_line","response_time_min","sla_target_response_min","response_vs_sla_pct"]]


,date,service_line,response_time_min,sla_target_response_min,response_vs_sla_pct
